# Sheet

[计算机体系结构基础](https://foxsen.github.io/archbase/)

[Making an OS (x86) Chapter 1 - CPU, Assembly, Booting](https://www.youtube.com/watch?v=MwPjvJ9ulSc)

[Write Your Own 64-bit Operating System Kernel #1 - Boot code and multiboot header](https://www.youtube.com/watch?v=FkrpUaGThTQ)

[【麻省理工学院】MIT 6.S081 操作系统工程 operating system engineering（中文字幕）](https://www.bilibili.com/video/BV1Dy4y1m7ZE/)

[MIT 6.S081](https://www.youtube.com/playlist?list=PLTsf9UeqkReZHXWY9yJvTwLJWYYPcKEqK)

[操作系统真相还原](https://zh.singlelogin.re/book/12340498/478394/%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F%E7%9C%9F%E8%B1%A1%E8%BF%98%E5%8E%9F.html)

[计算机操作系统（慕课版）](https://zh.singlelogin.re/book/27874348/f75301/%E8%AE%A1%E7%AE%97%E6%9C%BA%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F%E6%85%95%E8%AF%BE%E7%89%88.html)

# process

In [1]:
"""
进程调度
死锁
银行家算法
Banker's Algorithm
"""
def find_safe_sequence(available, max, allocation, need):
    work = available[:]
    finish = [False] * len(need)
    safe_sequence = []

    while len(safe_sequence) < len(need):
        for i, finished in enumerate(finish):
            if not finished:
                if all(need[i][j] <= work[j] for j in range(len(work))):
                    for j in range(len(work)):
                        work[j] += allocation[i][j]
                    finish[i] = True
                    safe_sequence.append(i)
                    break
        else:
            # 如果没有找到可以执行的进程，说明系统不安全
            return None
    return safe_sequence

def request_resources(process_id, request, available, allocation, need):
    if all(request[i] <= need[process_id][i] for i in range(len(request))) and all(request[i] <= available[i] for i in range(len(request))):
        # 试探性分配资源
        for i in range(len(request)):
            available[i] -= request[i]
            allocation[process_id][i] += request[i]
            need[process_id][i] -= request[i]

        # 检查这次分配后是否安全
        safe_sequence = find_safe_sequence(available, max, allocation, need)
        if safe_sequence is not None:
            # 如果安全，确认分配
            print(f"资源分配给P{process_id}后，系统是安全的。安全序列是: {safe_sequence}")
            return True
        else:
            # 如果不安全，回滚分配
            for i in range(len(request)):
                available[i] += request[i]
                allocation[process_id][i] -= request[i]
                need[process_id][i] += request[i]
            print(f"资源请求无法立即满足。P{process_id}的请求被拒绝。")
            return False
    else:
        print(f"P{process_id}的资源请求超过了它的需求或可用资源。请求被拒绝。")
        return False

# 可用资源
available = [3, 3, 2]
# 最大需求
max = [
    [7, 5, 3],
    [3, 2, 2],
    [9, 0, 2],
    [2, 2, 2],
    [4, 3, 3]
]
# 已分配资源
allocation = [
    [0, 1, 0],
    [2, 0, 0],
    [3, 0, 2],
    [2, 1, 1],
    [0, 0, 2]
]
# 还需要的资源
need = [
    [7, 4, 3],
    [1, 2, 2],
    [6, 0, 0],
    [0, 1, 1],
    [4, 3, 1]
]

# 测试请求资源
process_id = 1
request = [1, 0, 2]
request_resources(process_id, request, available, allocation, need)

资源分配给P1后，系统是安全的。安全序列是: [1, 3, 0, 2, 4]


True

In [ ]:
"""
信号量
进程互斥

初始化（Initialization）：设置信号量的初始值。
等待（Wait）或 P操作（Proberen，荷兰语“测试”）：如果信号量的值大于0，将其减1，然后继续执行；如果信号量的值为0，则进程被阻塞，直到信号量的值大于0。
信号（Signal）或 V操作（Verhogen，荷兰语“提高”）：将信号量的值加1，如果有进程因为等待信号量而被阻塞，它们将被唤醒。
"""
import threading

# 定义一个信号量
semaphore = threading.Semaphore(1)  # 二进制信号量，初始值为1

def critical_section():
    with semaphore:
        # 临界区代码，一次只允许一个线程进入
        print(f"Thread {threading.current_thread().name} entered the critical section.")
        # 模拟执行时间
        threading.sleep(2)
        print(f"Thread {threading.current_thread().name} leaving the critical section.")

# 创建线程
thread1 = threading.Thread(target=critical_section)
thread2 = threading.Thread(target=critical_section)

# 启动线程
thread1.start()
thread2.start()

# 等待线程结束
thread1.join()
thread2.join()

# page

In [ ]:
"""
先进先出（FIFO）算法：这是最简单的页面置换算法，它按照页面进入内存的顺序进行置换。最早进入内存的页面将最先被替换。FIFO算法实现简单，但可能会淘汰经常访问的页面，导致性能不佳
。

最近最久未使用（LRU）算法：LRU算法选择最近一段时间内最久没有被访问过的页面进行替换。这种算法基于时间局部性原理，即近期被访问的页面很可能在未来也会被访问。LRU算法性能较好，但实现起来相对复杂，需要记录每个页面的访问历史
。

最佳置换算法（OPT）：这是一种理论上的算法，它选择未来最长时间内不会被访问的页面进行置换。由于需要预知未来的页面访问情况，OPT算法在实际中无法实现，但可以作为衡量其他算法性能的标准
。

时钟（CLOCK）算法：CLOCK算法是LRU算法的一种近似实现，它通过一个循环链表来模拟时钟的工作。每个页面都有一个访问位，当页面被访问时，访问位被设置为1。在选择置换页面时，算法会循环检查链表中的页面，如果访问位为0，则选择该页面进行置换。如果为1，则将其置为0并继续检查下一个页面
。

最近未使用（NRU）算法：NRU算法是FIFO算法的改进，它通过跟踪页面的使用情况来决定页面是否应该被置换。NRU算法将页面分为几个类别，并优先置换那些既不经常被访问也没有被修改的页面
。

随机置换算法：随机算法简单地从内存中的页面中随机选择一个进行置换。这种方法简单但性能不稳定，因为它不考虑页面的使用历史或访问模式
。

最少使用（LFU）算法：LFU算法置换那些访问次数最少的页面。这种方法考虑了页面的访问频率，但可能不会很好地反映时间局部性原理
。

工作集算法：工作集算法基于这样的思想：一个进程在短期内的内存访问模式与它在长期内的访问模式相似。该算法会保留一个进程的工作集（即它在一段时间内访问的所有页面），并置换那些不在工作集中的页面

"""
class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  # 存储页面和其访问时间的字典
        self.access_time = 0  # 当前时间，用于模拟访问顺序

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1  # 如果页面不在缓存中，返回-1
        # 更新访问时间
        self.access_time += 1
        self.cache[key]['access_time'] = self.access_time
        return self.cache[key]['value']

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # 更新已存在的页面的访问时间和值
            self.access_time += 1
            self.cache[key]['access_time'] = self.access_time
            self.cache[key]['value'] = value
        else:
            if len(self.cache) >= self.capacity:
                # 如果缓存已满，移除最久未使用的页面
                self._remove_least_recently_used()
            self.access_time += 1
            # 添加新页面到缓存
            self.cache[key] = {'access_time': self.access_time, 'value': value}

    def _remove_least_recently_used(self) -> None:
        # 移除访问时间最小的页面，即最久未使用的页面
        least_recently_used_key = min(self.cache, key=lambda k: self.cache[k]['access_time'])
        del self.cache[least_recently_used_key]

# 示例使用
lru_cache = LRUCache(2)
lru_cache.put(1, 1)  # 缓存是 {1=1}
lru_cache.put(2, 2)  # 缓存是 {1=1, 2=2}
print(lru_cache.get(1))  # 返回 1，缓存是 {2=2, 1=1}
lru_cache.put(3, 3)  # 移除最久未使用的页面 2，缓存是 {1=1, 3=3}
print(lru_cache.get(2))  # 返回 -1（未找到）
lru_cache.put(4, 4)  # 移除最久未使用的页面 1，缓存是 {3=3, 4=4}
print(lru_cache.get(1))  # 返回 -1（未找到）
print(lru_cache.get(3))  # 返回 3
print(lru_cache.get(4))  # 返回 4

# I/O

In [ ]:
"""
先来先服务（FCFS, First-Come, First-Served）：

按照磁盘请求到达的顺序进行服务。
简单易实现，但可能导致“饥饿”现象，即请求到达较晚的磁盘I/O操作可能会等待很长时间。
最短寻道时间优先（SSTF, Shortest Seek Time First）：

选择当前磁头位置最近的磁盘请求。
可以提供比FCFS更好的平均寻道时间，但可能导致“饥饿”现象。
扫描（SCAN）：

磁头从一个方向开始移动，直到达到最后一个请求，然后反转方向。
可以减少寻道时间，但可能不是最优的，因为它不考虑磁盘请求的到达顺序。
循环扫描（C-SCAN）：

磁头在一个方向上移动，服务所有请求，到达最后一个请求后直接跳转到起始位置，而不是反向扫描。
可以避免“饥饿”现象，因为每个请求都会被服务。
最近最久未使用（NRU, Not Recently Used）：

类似于LRU页面置换算法，选择最长时间未被访问的磁盘块进行调度。
适用于某些特定场景，不如SCAN和C-SCAN常用。
LOOK（C-LOOK）：

一种改进的C-SCAN算法，磁头在到达请求的末端后，立即返回到起始位置，而不是等待。
可以提高磁盘调度的效率。
C-LOOK：

与LOOK类似，但是磁头在完成一个方向上的请求后，会立即跳转到起始位置，而不是等待。
公平磁盘调度（FDD, Fair Disk Scheduling）：

为每个进程分配一个访问磁盘的时间段，确保所有进程公平地访问磁盘。
可以避免某些进程长时间占用磁盘资源。
跳跃扫描（SKIP SCAN）：

将磁盘请求分成多个组，磁头在每个组内顺序服务请求，然后跳到下一个组。
旨在减少寻道时间，同时考虑了磁盘请求的局部性。
磁盘区域分组（Zoning）：

将磁盘分割成多个区域，每个区域由一个特定的磁头处理。
可以提高磁盘的并行处理能力。
"""

class ScanDiskScheduling:
    def __init__(self, requests, head_start):
        self.requests = sorted(requests)  # 磁盘请求列表，已排序
        self.head_start = head_start  # 磁头起始位置
        self.direction = 1  # 移动方向，1表示向右，-1表示向左

    def scan(self):
        service_order = []
        current_position = self.head_start
        while self.requests:
            if self.requests[0] >= current_position:
                service_order.append(self.requests.pop(0))
                current_position = self.requests[0] if self.requests else current_position
            else:
                if self.requests[-1] < current_position:
                    self.direction = -1
                service_order.append(self.requests.pop())
                current_position = self.requests[-1] if self.requests else current_position
        return service_order

# 示例使用
requests = [98, 183, 37, 122, 14, 124, 65, 67]
head_start = 37
scan_scheduler = ScanDiskScheduling(requests, head_start)
service_order = scan_scheduler.scan()
print("服务顺序:", service_order)

# file

# program

## 状态机

Stack Frame

PC program counter

二进制 程序

= M 内存 + R 寄存器

= 计算 + syscall


# thread